In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow_datasets as tfds
import numpy as np

# 9.

In [61]:
train_set, valid_set, test_set = tfds.load(
    'mnist',
    split=['train[:50000]', 'train[50000:]', 'test'],
    as_supervised=True,
    shuffle_files=True,
)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
class RBFLayer(layers.Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff,2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

In [ ]:
def normalize(image):
    return tf.cast(image, tf.float32) / 255.


model = keras.models.Sequential([
    layers.Input((28, 28, 1)),
    layers.Lambda(normalize),
    layers.ZeroPadding2D(padding=2),
    layers.Conv2D(filters=6, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=16, kernel_size=5, activation='tanh'),
    layers.AveragePooling2D(pool_size=2, strides=[2, 2]),
    layers.Conv2D(filters=120, kernel_size=5, activation='tanh'),
    layers.Flatten(),
    layers.Dense(84, activation='tanh'),
    RBFLayer(10, 0.5)
])
model.compile(
    optimizer='nadam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
batch_size = 128
train_set = train_set.batch(batch_size).prefetch(1)
test_set = test_set.batch(batch_size).prefetch(1)
valid_set = valid_set.batch(batch_size).prefetch(1)

In [ ]:
model.fit(
    train_set,
    epochs=10,
    validation_data=valid_set
)
model.evaluate(test_set)

Epoch 1/10
391/391 [==============================] - 9s 15ms/step - loss: 0.4904 - accuracy: 0.8854 - val_loss: 0.1457 - val_accuracy: 0.9587
Epoch 2/10
391/391 [==============================] - 3s 7ms/step - loss: 0.1089 - accuracy: 0.9694 - val_loss: 0.0850 - val_accuracy: 0.9762
Epoch 3/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0695 - accuracy: 0.9803 - val_loss: 0.0718 - val_accuracy: 0.9787
Epoch 4/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0527 - accuracy: 0.9854 - val_loss: 0.0651 - val_accuracy: 0.9805
Epoch 5/10
391/391 [==============================] - 3s 6ms/step - loss: 0.0424 - accuracy: 0.9881 - val_loss: 0.0602 - val_accuracy: 0.9810
Epoch 6/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0346 - accuracy: 0.9907 - val_loss: 0.0568 - val_accuracy: 0.9823
Epoch 7/10
391/391 [==============================] - 3s 7ms/step - loss: 0.0287 - accuracy: 0.9925 - val_loss: 0.0544 - val_accuracy: 0.9838
Epoch

[0.04711465537548065, 0.9868000149726868]

# 10.

In [2]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, (224, 224))
    final_image = keras.applications.vgg19.preprocess_input(resized_image)
    return final_image, label


def augment(dataset, length):
    augmentations = [
        lambda image: tf.image.flip_left_right(image),
        lambda image: tf.image.flip_up_down(image),
        lambda image: tf.image.stateless_random_contrast(
            image, 0.2, 0.5, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_saturation(
            image, 0.5, 1.0, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_brightness(
            image, 0.2, seed=(42, 42)
        ),
        lambda image: tf.image.stateless_random_crop(
            image, size=(1, 2, 3), seed=(42, 42)
        )
    ]
    augmented_sets = []
    for augmentation in augmentations:
        augmented_sets.append(dataset.map(
            lambda image, label: (augmentation(image), label)
        ))

    updated_dataset = dataset
    for augmented_set in augmented_sets:
        updated_dataset = updated_dataset.concatenate(augmented_set)

    new_length = length*(len(augmented_sets)+1)
    return updated_dataset, new_length


In [3]:
K.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

(train_set1, train_set2, valid_set, test_set), ds_info = tfds.load(
    'oxford_flowers102',
    split=['train[:50%]', 'train[50%:]', 'validation', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True
)
train_len1 = ds_info.splits['train[:50%]'].num_examples
train_len2 = ds_info.splits['train[50%:]'].num_examples
n_classes = ds_info.features['label'].num_classes
batch_size = 32
train_set1, train_len1 = augment(train_set1, train_len1)
train_set1 = train_set1.map(preprocess).shuffle(train_len1).batch(batch_size).prefetch(1)
train_set2, train_len2 = augment(train_set2, train_len2)
train_set2 = train_set2.map(preprocess).shuffle(train_len2).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

In [4]:
base_model = keras.applications.VGG19(weights='imagenet', include_top=False)

In [5]:
for layer in base_model.layers:
    print(layer)

In [5]:
avg = layers.GlobalAveragePooling2D()(base_model.output)
output = layers.Dense(n_classes, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
K.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

for layer in base_model.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# TODO: implement generator
for _ in range(20):
    model.fit(train_set1, epochs=1, validation_data=valid_set)
    model.fit(train_set2, epochs=1, validation_data=valid_set)
    train_set1 = train_set1.shuffle(train_len1)
    train_set2 = train_set2.shuffle(train_len2)


112/112 [==============================] - 47s 255ms/step - loss: 2.9632 - accuracy: 0.8594 - val_loss: 47.7350 - val_accuracy: 0.7520


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cd gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/chapter11

In [ ]:
model.save('mymodel.keras')

In [ ]:
model = keras.models.load_model('mymodel.keras')

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set, epochs=20, validation_data=valid_set)